In [1]:
import numpy as np
import scipy.stats
import scs

In [2]:
#############################################
#      Generate random cone problems        #
#############################################

def pos(x):
    return (x + np.abs(x)) / 2.
    
def gen_feasible(m, n, p_scale = 0.1):
    z = np.random.randn(m)
    y = pos(z)
    s = y - z

    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    # Make problem slightly more numerically challenging:
    A = np.random.randn(m, n)
    U, S, V = np.linalg.svd(A, full_matrices=False)
    S = S**2
    S /= np.max(S)
    A = (U * S) @ V
        
    x = np.random.randn(n)
    c = -A.T @ y - P @ x
    b = A.dot(x) + s
    
    b /= np.linalg.norm(b)
    c /= np.linalg.norm(c)

    return (P, A, b, c)

In [3]:
def is_optimal(P, A, b, c, x, y, tol=1e-6):
    s = b - A @ x
    
    if (np.linalg.norm(P @ x + c + A.T @ y) < tol and
        np.abs(y.T @ s) < tol and
        np.linalg.norm(s - pos(s)) < tol and
        np.linalg.norm(y - pos(y)) < tol):
        return True
    return False

In [4]:
# ''linear'' projection logic

class LinearProjector(object):
    def __init__(self, P, A, b, c):
        (m,n) = A.shape
        self.A = A
        self.h = np.hstack((c, b))        
        self.L = scipy.linalg.cho_factor(np.eye(n) + P + A.T @ A)
        self.g = self._solve(self.h)

    def _solve(self, v):
        (m, n) = self.A.shape
        sol = np.zeros(n+m,)
        sol[:n] = scipy.linalg.cho_solve(self.L, v[:n] - self.A.T @ v[n:])
        sol[n:] = v[n:] + self.A @ sol[:n]
        return sol
        
    def project(self, w):
        g = self.g
        p = self._solve(w[:-1])
        
        _a = 1 + g.T@g
        _b = w[:-1].T @ g - 2 * p.T @ g - w[-1]
        _c = p.T@p - w[:-1].T @ p
        
        tau = (-_b + np.sqrt(_b ** 2 - 4 * _a * _c)) / 2 / _a
        
        return np.hstack((p - tau * g, tau))

# Douglas-Rachford splitting / ADMM for QP
def solve_qp_dr(P, A, b, c, N=1000):
    (m,n) = np.shape(A)

    lp = LinearProjector(P, A, b, c)

    u = np.zeros(n+m+1,)
    #u[:n+m] = np.hstack((c, b))
    u[-1] = 1.

    def proj_cone(v):
        v[n:] = np.maximum(v[n:], 0.)
        return v


    use_dr = True # slightly different DR vs ADMM
    
    for i in range(N):
        # for DR ut converges to sol *not* u, see Patrinos, Stella, Bemporad, 2014
        # v - ut go to zero
        dr_lam = 0.5 # \in [0,1], 0.5 = DR, 1.0 = PR

        ut = lp.project(u)
        v = proj_cone(2 * ut - u)
        u += 2 * dr_lam * (v - ut)
        
        x = ut[:n] / ut[-1]
        y = ut[n:-1] / ut[-1]
        
        if is_optimal(P, A, b, c, x, y, tol=1e-9):
            break
        
    print(i)
    return x,y



In [5]:
m = 15
n = 10
N = int(5e3)

In [10]:
feas_h_is = []
feas_nh_is = []

conds = []

np.random.seed(123)

(P, A, b, c) = gen_feasible(m, n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)
print(y)

514
[-0.13351969  0.09140453  0.11516882  0.28373837 -0.07264948 -0.2798121
 -0.25180523  0.30405864 -0.13665965 -0.25222408]
[ 3.54744623e-10  1.46516000e-01  3.08666010e-10  8.37947276e-12
  1.13901231e-10  2.93315538e-01 -6.08878503e-11 -1.31061643e-10
  2.08074169e-01 -1.85143373e-10  1.06664439e-10 -1.35809290e-10
 -4.09269196e-11 -4.27421504e-10 -1.45058328e-10]


In [7]:
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
#probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c, normalize=False, iters=-1)

cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 0)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 150
eps = 1.00e-05, alpha = 1.00, max_iters = 5000, normalize = 0
acceleration_lookback = 0, rho_x = 1.00e+00
Variables n = 10, constraints m = 15
Cones:	linear vars: 15
Setup time: 6.57e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 8.78e+18  1.24e+19  9.20e-01 -1.01e+19  4.92e+15  1.87e+19  6.08e-03 
   100| 1.46e-04  3.01e-04  1.19e-04 -4.35e-01 -2.76e-02  1.00e+00  6.57e-03 
   200| 2.39e-06  4.74e-06  2.58e-06 -4.35e-01 -2.74e-02  1.00e+00  7.01e-03 
------------------------------------------------------------------

{'x': array([-0.13352833,  0.0913651 ,  0.11523738,  0.28381225, -0.07275692,
        -0.27992132, -0.25184019,  0.30407561, -0.13666681, -0.25228216]),
 'y': array([0.        , 0.14644878, 0.        , 0.        , 0.        ,
        0.2933768 , 0.        , 0.        , 0.20807256, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 's': array([0.27185662, 0.14642033, 0.00996228, 0.46548158, 0.17484812,
        0.29338444, 0.74167935, 0.11942951, 0.20807279, 0.25759663,
        0.21707643, 0.0081178 , 0.00399309, 0.17121053, 0.18987706]),
 'info': {'statusVal': 1,
  'iter': 200,
  'pobj': -0.4348102477000499,
  'dobj': -0.027402283703497673,
  'resPri': 2.394145627063584e-06,
  'resDual': 4.741664119959889e-06,
  'relGap': 2.5772108645824504e-06,
  'resInfeas': nan,
  'resUnbdd': 2.2998615969244023,
  'solveTime': 7.103553,
  'setupTime': 6.565199,
  'status': 'Solved'}}

In [8]:
(P, A, b, c) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)

864
[-0.01332149 -0.19445579 -0.07097391 -0.28368241  0.17437659  0.18742115
 -0.29593732 -0.37628333  0.10179457  0.20877669]


In [9]:
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 0)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 0)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 150
eps = 1.00e-05, alpha = 1.00, max_iters = 5000, normalize = 0
acceleration_lookback = 0, rho_x = 1.00e+00
Variables n = 10, constraints m = 15
Cones:	linear vars: 15
Setup time: 2.83e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 7.89e+18  5.30e+17  9.64e-01 -2.05e+19 -3.80e+17  2.26e+19  4.46e-03 
   100| 1.87e-03  5.79e-03  2.82e-03 -3.82e-01 -3.77e-01  1.00e+00  5.72e-03 
   200| 4.02e-04  1.02e-03  7.38e-05 -3.80e-01 -3.80e-01  1.00e+00  1.01e-02 
   300| 4.12e-05  1.23e-04  4.37e-05 -3.81e-01 -3.81e-01  1.00e+00

{'x': array([-0.01334372, -0.19445053, -0.07087348, -0.2836757 ,  0.1743111 ,
         0.18741447, -0.29588919, -0.37625044,  0.10179928,  0.2087704 ]),
 'y': array([0.        , 2.2412773 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.22115604, 0.        , 0.49274638]),
 's': array([0.16160839, 2.24129238, 0.32458216, 0.1674836 , 0.09199965,
        0.30142041, 0.29605914, 0.28286736, 0.16309568, 0.11625841,
        0.19725877, 0.49202194, 2.22113606, 0.35007926, 0.49274893]),
 'info': {'statusVal': 1,
  'iter': 400,
  'pobj': -0.38062722757524825,
  'dobj': -0.3806259328460403,
  'resPri': 4.008644826836244e-06,
  'resDual': 9.162336611818112e-06,
  'relGap': 7.351181744066344e-07,
  'resInfeas': nan,
  'resUnbdd': 2.6272340363154933,
  'solveTime': 10.998455,
  'setupTime': 7.096552,
  'status': 'Solved'}}